# SFT with QLoRA - OPTIMIZED FOR KAGGLE

**Goals:** 75-80% cost reduction, 60-70% time reduction

**Optimizations:**
- QLoRA (4-bit quantization) for 3-4x speedup
- 30% dataset sampling for faster training
- BF16 precision (stable, no gradient scaling issues)
- 2 epochs instead of 3
- Estimated time: 3-5 hours ✅

In [ ]:
# Install with bitsandbytes for QLoRA support
!pip install -q transformers datasets peft trl accelerate wandb evaluate bitsandbytes scipy

In [ ]:
import sys; sys.path.append('../')
import torch, time, json, os
from src.data.loader import DatasetLoader
from src.data.processor import DataProcessor
from src.model.base import load_base_model, load_base_model_qlora, load_tokenizer, get_model_info
from src.model.lora import LoRAWrapper
from src.training.sft import SFTTrainer
from src.utils.metrics import MetricsTracker

# Optional wandb - no API key required
try:
    import wandb
    wandb.init(project='persona-chatbot-rlhf', name='sft-qlora', mode='disabled')  # offline mode
    USE_WANDB = True
except:
    USE_WANDB = False
    class wandb:
        @staticmethod
        def log(*args, **kwargs): pass
        @staticmethod
        def finish(): pass

print(f'W&B: {"enabled (offline)" if USE_WANDB else "disabled"}')
print(f'GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"}')
print(f'CUDA Available: {torch.cuda.is_available()}')

In [ ]:
# Config - OPTIMIZED FOR KAGGLE (3-5 hour runtime)
model_config = {
    'name': 'gpt2-medium',
    'cache_dir': '../models/base',
    'device_map': 'auto',
    'use_qlora': True  # Enable QLoRA for 3-4x speedup
}

# LoRA config - Optimized for speed (r=8 instead of 16)
lora_config = {
    'r': 8,                    # Reduced from 16 (faster, still effective)
    'alpha': 16,               # Reduced from 32 (matched to r)
    'dropout': 0.05,           # Reduced from 0.1
    'target_modules': ['c_attn'],  # Only attention (faster than c_attn + c_proj)
    'bias': 'none',
    'task_type': 'CAUSAL_LM'
}

# Training config - Optimized for Kaggle 9-12h limit
training_config = {
    'output_dir': '../models/sft',
    'num_epochs': 2,                 # 3 → 2 (1.5x faster)
    'max_steps': 3000,               # Hard limit for safety
    'per_device_batch_size': 8,      # 4 → 8 (QLoRA saves memory)
    'gradient_accumulation_steps': 4,  # Effective batch = 32
    'learning_rate': 2e-4,
    'warmup_steps': 300,             # 500 → 300 (fewer steps needed)
    'weight_decay': 0.01,
    'bf16': True,                    # CRITICAL: Use BF16 not FP16 (prevents precision errors)
    'gradient_checkpointing': True,  # Memory efficient
    'logging_steps': 50,
    'eval_steps': 300,               # Must be divisor of save_steps               # 500 → 300 (more frequent)
    'save_steps': 600,               # Must be multiple of eval_steps (for load_best_model_at_end)               # 1000 → 500
    'save_total_limit': 2,           # Keep only 2 checkpoints
    'dataloader_num_workers': 2,     # Parallel data loading
    'optim': 'paged_adamw_8bit',     # 8-bit optimizer for QLoRA
    'use_wandb': False
}

data_config = {
    'base_model': 'gpt2-medium',
    'max_length': 512,
    'dataset_fraction': 0.3  # Use 30% for faster training (can increase to 0.5 if needed)
}

print('Configuration loaded:')
print(f'  - QLoRA: {model_config["use_qlora"]}')
print(f'  - Dataset: {data_config["dataset_fraction"]*100}%')
print(f'  - Precision: BF16 (safe, no gradient scaling)')
print(f'  - Max steps: {training_config["max_steps"]}')
print(f'  - Estimated time: 3-5 hours')

In [ ]:
# Load data - OPTIMIZED: Use 30% subset
print('Loading dataset...')
loader = DatasetLoader()
train_full = loader.load_personachat(split='train', use_synthetic=True)
val_full = loader.load_personachat(split='validation', use_synthetic=True)

# Sample dataset for faster training
dataset_fraction = data_config.get('dataset_fraction', 0.3)
train_size = int(len(train_full) * dataset_fraction)
val_size = int(len(val_full) * dataset_fraction)

print(f'\nSampling {dataset_fraction*100}% of dataset:')
print(f'  Train: {len(train_full)} → {train_size} examples')
print(f'  Val: {len(val_full)} → {val_size} examples')

train = train_full.select(range(train_size))
val = val_full.select(range(val_size))

# Preprocess and tokenize
print('\nPreprocessing...')
processor = DataProcessor(config=data_config)
train_proc = processor.tokenize(processor.preprocess(train))
val_proc = processor.tokenize(processor.preprocess(val))
print(f'Processed: {len(train_proc)} train, {len(val_proc)} val examples')

In [ ]:
# Model + QLoRA - OPTIMIZED VERSION
print('\nLoading model...')

# Load with QLoRA if enabled (3-4x faster than regular LoRA)
if model_config.get('use_qlora', False):
    print('Using QLoRA (4-bit quantization):')
    model = load_base_model_qlora(model_config)
else:
    print('Using regular LoRA (slower):')
    model = load_base_model(model_config)

tokenizer = load_tokenizer(model_config)
total_p = get_model_info(model)['total_parameters']

# Apply LoRA
print('\nApplying LoRA adapters...')
lora = LoRAWrapper(lora_config)
model = lora.apply_lora(model, lora_config)
lora.print_trainable_params(model)

trainable_p = get_model_info(model)['trainable_parameters']
reduction = (1 - trainable_p/total_p) * 100

print(f'\nModel statistics:')
print(f'  Total params: {total_p/1e6:.0f}M')
print(f'  Trainable params: {trainable_p/1e6:.1f}M')
print(f'  Reduction: {reduction:.1f}%')
if model_config.get('use_qlora', False):
    print(f'  Memory savings (QLoRA): ~75%')
    print(f'  Speed improvement: 3-4x faster')

In [ ]:
# Train with optimized settings
print('\nStarting training...')
print(f'  Epochs: {training_config["num_epochs"]}')
print(f'  Batch size: {training_config["per_device_batch_size"]}')
print(f'  Gradient accumulation: {training_config["gradient_accumulation_steps"]}')
print(f'  Effective batch: {training_config["per_device_batch_size"] * training_config["gradient_accumulation_steps"]}')
print(f'  Max steps: {training_config["max_steps"]}')
print(f'  Precision: BF16 (safe, no gradient scaling issues)')
print(f'\nEstimated steps: ~{(len(train_proc) // (training_config["per_device_batch_size"] * training_config["gradient_accumulation_steps"])) * training_config["num_epochs"]}')
print(f'Estimated time: 3-5 hours\n')

tracker = MetricsTracker(gpu_hourly_rate=0.35)
tracker.start_timing()
os.makedirs(training_config['output_dir'], exist_ok=True)

trainer = SFTTrainer(model, tokenizer, train_proc, val_proc, training_config)
results = trainer.train()

print('\n✅ Training complete!')

In [ ]:
# Metrics and validation
hours = tracker.stop_timing() / 3600
gpus = torch.cuda.device_count() if torch.cuda.is_available() else 1
cost = tracker.track_cost(hours, gpus)
savings = tracker.calculate_savings('full_finetuning', 'lora')

print(f'\nTraining Results:')
print(f'  Time: {hours:.2f} hours')
print(f'  Cost: ${cost:.2f}')
print(f'\nSavings vs Full Fine-tuning:')
print(f'  Time: {savings["time_savings_percent"]:.1f}% (target: 60-70%)')
print(f'  Cost: {savings["cost_savings_percent"]:.1f}% (target: 75-80%)')

cost_ok = savings['cost_savings_percent'] >= 75
time_ok = savings['time_savings_percent'] >= 60

print(f'\nTargets Met:')
print(f'  Cost: {"✅" if cost_ok else "❌"}  {savings["cost_savings_percent"]:.1f}% >= 75%')
print(f'  Time: {"✅" if time_ok else "❌"}  {savings["time_savings_percent"]:.1f}% >= 60%')

wandb.log({
    'hours': hours,
    'cost': cost,
    'time_savings%': savings['time_savings_percent'],
    'cost_savings%': savings['cost_savings_percent'],
    'qlora_enabled': model_config.get('use_qlora', False)
})

In [ ]:
# Save model and summary
print('\nSaving model...')
path = f"{training_config['output_dir']}/final"
os.makedirs(path, exist_ok=True)
model.save_pretrained(path)
tokenizer.save_pretrained(path)

summary = {
    'params': int(trainable_p),
    'hours': hours,
    'cost': cost,
    'savings_time%': savings['time_savings_percent'],
    'savings_cost%': savings['cost_savings_percent'],
    'targets_met': {'cost': cost_ok, 'time': time_ok},
    'config': {
        'qlora_enabled': model_config.get('use_qlora', False),
        'dataset_fraction': data_config.get('dataset_fraction', 1.0),
        'precision': 'BF16' if training_config.get('bf16') else 'FP32'
    }
}

with open(f"{training_config['output_dir']}/summary.json", 'w') as f:
    json.dump(summary, f, indent=2)

wandb.finish()

print(f'\n✅ Model saved to {path}')
print(f'✅ Summary saved to {training_config["output_dir"]}/summary.json')
print(f'\n🎉 SFT training complete! Next: 4_reward_and_ppo.ipynb')